In [1]:
import numpy as np
import scipy as sp
import igl
import trimesh
import polyscope as ps
import os

from cotanLaplace import *
from massmatrix import *
from Mollification import *

root_folder = os.getcwd()

### TODO? Make the demos interactive (e.g. select source vertices for heat geodisics)

In [2]:
# [V,F]= igl.read_triangle_mesh(os.path.join(
#              root_folder, "../data/bad_triangle/", "holly_wand_complete_letter_A_Print.stl"))
# up_direction = 'z_up'

[V,F]= igl.read_triangle_mesh(os.path.join(
            root_folder, "../data", "BadConeSeg.obj"))
up_direction = 'y_up'

#[V,F]= igl.read_triangle_mesh(os.path.join(
#            root_folder, "../data", "spot.obj"))
#up_direction = 'z_up'

In [3]:
#init poly scope and show mesh
ps.init()
ps.set_up_dir(up_direction)
ps.set_ground_plane_mode('tile_reflection')
#register mesh
#mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0], edge_width=0.9, edge_color=[0.2, 0.1, 0.8, 1.0])
#ps.show()

# Solve Poisson equation (interpolation)

In [4]:
# use cotan Laplace and Mass Matrix to solve Lx = -M(rho - rho0)

avg_edge_length = igl.avg_edge_length(V, F)
delta = 0.00000000000000 * avg_edge_length
E, eps, newL = IntrinsicMollificationConstant(V, F, delta)
L = cotanLaplace(F, newL, neg_hack=NEG_HACK.TO_ABS, nan_hack=NAN_HACK.TO_ZERO)
M = massmatrix(newL, F, MASSMATRIX_TYPE.BARYCENTRIC)

# choose 2 random vertex indices
v1 = 593203 % V.shape[0]
v2 = 131394 % V.shape[0]

# rho is a column vector of length V.shape[0] and with all zeros except for rho[v1] and rho[v2]
rho = sp.zeros((1, V.shape[0]))
rho[:,v1] = 100
rho[:,v2] = -100

def solvePoisson(L, M, rho):
        totArea = sp.sum(M.diagonal())

        # rho0 is a column vector of length V.shape[0], with all the same value = sum(rho[i] * M[i,i]) / totArea
        rho0 = sp.sum(rho * M.diagonal()) / totArea # this is more general than just summing at the 2 vertices

        f = sp.sparse.linalg.spsolve(L, -M.dot(rho.T - rho0.T))

        return f

f = solvePoisson(L, M, rho)

# plot the result
mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0])
mesh.add_scalar_quantity("f", 
        f, defined_on='vertices', cmap="viridis")

ps.show()




2 2 Neg
4 2 Neg
6 2 Neg
8 2 Neg
10 2 Neg
12 2 Neg
14 2 Neg
15 1 Neg
17 1 Neg
19 1 Neg
21 1 Neg
23 1 Neg
25 1 Neg
27 1 Neg
29 0 Nan
29 1 Nan
29 2 Nan
33 2 Neg
35 2 Neg
37 2 Neg
39 2 Neg
41 2 Neg
43 2 Neg
45 2 Neg
46 1 Neg
48 1 Neg
50 1 Neg
52 1 Neg
54 1 Neg
56 1 Neg
58 1 Neg
60 0 Nan
60 1 Nan
60 2 Nan
64 2 Neg
66 2 Neg
68 2 Neg
70 2 Neg
72 2 Neg
74 2 Neg
76 2 Neg
77 1 Neg
79 1 Neg
81 1 Neg
83 1 Neg
85 1 Neg
87 1 Neg
89 1 Neg
91 0 Nan
91 1 Nan
91 2 Nan
95 2 Neg
97 2 Neg
99 2 Neg
101 2 Neg
103 2 Neg
105 2 Neg
107 2 Neg
108 1 Neg
110 1 Neg
112 1 Neg
114 1 Neg
116 1 Neg
118 1 Neg
120 1 Neg
122 0 Nan
122 1 Nan
122 2 Nan
126 2 Neg
128 2 Neg
130 2 Neg
132 2 Neg
134 2 Neg
136 2 Neg
138 2 Neg
139 1 Neg
141 1 Neg
143 1 Neg
145 1 Neg
147 1 Neg
149 1 Neg
151 1 Neg
153 0 Nan
153 1 Nan
153 2 Nan
157 2 Neg
159 2 Neg
161 2 Neg
163 2 Neg
165 2 Neg
167 2 Neg
169 2 Neg
170 1 Neg
172 1 Neg
174 1 Neg
176 1 Neg
178 1 Neg
180 1 Neg
182 1 Neg
184 0 Nan
184 1 Nan
184 2 Nan
188 2 Neg
190 2 Neg
192 2 Neg
194 2 Ne

d:\4B\intrinsic_mollification\src\cotanLaplace.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  wjk = (lengths[j] ** 2 + lengths[k] ** 2 - lengths[i] ** 2) / (8*area)
d:\4B\intrinsic_mollification\src\cotanLaplace.py:54: RuntimeWarning: invalid value encountered in scalar divide
  wjk = (lengths[j] ** 2 + lengths[k] ** 2 - lengths[i] ** 2) / (8*area)


 1 Neg
951 1 Neg
953 1 Neg
955 1 Neg
957 1 Neg
959 0 Nan
959 1 Nan
959 2 Nan
963 2 Neg
965 2 Neg
967 2 Neg
969 2 Neg
971 2 Neg
973 2 Neg
975 2 Neg
976 1 Neg
978 1 Neg
980 1 Neg
982 1 Neg
984 1 Neg
986 1 Neg
988 1 Neg
990 0 Nan
990 1 Nan
990 2 Nan
994 2 Neg
996 2 Neg
998 2 Neg
1000 2 Neg
1002 2 Neg
1004 2 Neg
1006 2 Neg
1007 1 Neg
1009 1 Neg
1011 1 Neg
1013 1 Neg
1015 1 Neg
1017 1 Neg
1019 1 Neg
1021 0 Nan
1021 1 Nan
1021 2 Nan
1025 2 Neg
1027 2 Neg
1029 2 Neg
1031 2 Neg
1033 2 Neg
1035 2 Neg
1037 2 Neg
1038 1 Neg
1040 1 Neg
1042 1 Neg
1044 1 Neg
1046 1 Neg
1048 1 Neg
1050 1 Neg
1052 0 Nan
1052 1 Nan
1052 2 Nan
1056 2 Neg
1058 2 Neg
1060 2 Neg
1062 2 Neg
1064 2 Neg
1066 2 Neg
1068 2 Neg
1069 1 Neg
1071 1 Neg
1073 1 Neg
1075 1 Neg
1077 1 Neg
1079 1 Neg
1081 1 Neg
1083 0 Nan
1083 1 Nan
1083 2 Nan
1087 2 Neg
1089 2 Neg
1091 2 Neg
1093 2 Neg
1095 2 Neg
1097 2 Neg
1099 2 Neg
1100 1 Neg
1102 1 Neg
1104 1 Neg
1106 1 Neg
1108 1 Neg
1110 1 Neg
1112 1 Neg
1114 0 Nan
1114 1 Nan
1114 2 Nan
1118 2 N

C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_23120/3731582627.py:14: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead
  rho = sp.zeros((1, V.shape[0]))
C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_23120/3731582627.py:19: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  totArea = sp.sum(M.diagonal())
C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_23120/3731582627.py:22: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  rho0 = sp.sum(rho * M.diagonal()) / totArea # this is more general than just summing at the 2 vertices
c:\Python310\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


## Mesh Smoothing

In [22]:
# use cotan Laplace matrix for smoothing
# solving (M - h * L) Vnew = M * Vold

mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0])
avg_edge_length = igl.avg_edge_length(V, F)
delta = 0.0000000001 * avg_edge_length
E, eps, newL = IntrinsicMollificationConstant(V, F, delta)
L = cotanLaplace(F, newL, neg_hack=NEG_HACK.NONE, nan_hack=NAN_HACK.NONE)

#L = igl.cotmatrix(V, F)
h = 0.5 * avg_edge_length

t = 0
n = 100

def smooth():
    global t
    global n
    global V
    global mesh

    if t < n:
        newL = igl.edge_lengths(V, F)
        M = massmatrix(newL, F, MASSMATRIX_TYPE.BARYCENTRIC)
        #M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)
        #M = igl.massmatrix_intrinsic(newL, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)
        A = M - h * L

        for i in range(3):
            V[:,i] = sp.sparse.linalg.spsolve(A, M.dot(V[:,i])) # Todo look for a way to use a positive semi-definite solver

        mesh.update_vertex_positions(V)
        t += 1
        print("Smoothing iteration: ", t)

ps.set_user_callback(smooth)

ps.show()

ps.clear_user_callback()



Smoothing iteration:  1
Smoothing iteration:  2
Smoothing iteration:  3
Smoothing iteration:  4
Smoothing iteration:  5
Smoothing iteration:  6
Smoothing iteration:  7
Smoothing iteration:  8
Smoothing iteration:  9
Smoothing iteration:  10
Smoothing iteration:  11
Smoothing iteration:  12
Smoothing iteration:  13
Smoothing iteration:  14
Smoothing iteration:  15
Smoothing iteration:  16
Smoothing iteration:  17
Smoothing iteration:  18
Smoothing iteration:  19
Smoothing iteration:  20
Smoothing iteration:  21
Smoothing iteration:  22
Smoothing iteration:  23
Smoothing iteration:  24
Smoothing iteration:  25
Smoothing iteration:  26
Smoothing iteration:  27
Smoothing iteration:  28
Smoothing iteration:  29
Smoothing iteration:  30
Smoothing iteration:  31
Smoothing iteration:  32
Smoothing iteration:  33
Smoothing iteration:  34
Smoothing iteration:  35
Smoothing iteration:  36
Smoothing iteration:  37
Smoothing iteration:  38
Smoothing iteration:  39
Smoothing iteration:  40
Smoothing

## Next: Heat Method for Geodesic Distances

In [27]:
# L = igl.cotmatrix(V, F)
# M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
# h = 0.005
# A = M + h * L

# print(sp.sparse.linalg.norm(M))
# I = sp.sparse.identity(M.shape[0])
# print(I.dot(V[:, 0]) - V[:, 0])
# print(sp.sparse.linalg.norm(sp.sparse.identity(M.shape[0])))


0.08031036303393863
[0. 0. 0. ... 0. 0. 0.]
119.54078801814885


In [12]:
L = igl.edge_lengths(V, F)         # columns correspond to edges lengths [1,2],[2,0],[0,1]

M_ours_i = massmatrix(L, F, MASSMATRIX_TYPE.BARYCENTRIC).todense()
M_igl = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC).todense()
M_igl_i = igl.massmatrix_intrinsic(L, F, igl.MASSMATRIX_TYPE_BARYCENTRIC).todense()
print(np.linalg.norm(M_ours_i - M_igl_i))
print(np.linalg.norm(M_igl - M_igl_i))


6.928722052989442e-18
0.0
